<a href="https://colab.research.google.com/github/kappakpr/DeepLearning/blob/master/senti2_GBM_H2O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


In [2]:
!pip install h2o==3.30.0.1
#can also install without specifying the version, pip will install the latest version available

     |████████████████████████████████| 129.5MB 80kB/s 
  Created wheel for h2o: filename=h2o-3.30.0.1-py2.py3-none-any.whl size=129557718 sha256=91f71f5280cfd85bbfc5dcfa009c9fd9ae823a46e0c5eea7fc0c700700ea3284
  Stored in directory: /root/.cache/pip/wheels/db/42/63/d118d7218432054da842f5d10bd4f99e978bde0931164e6c7a
Successfully built h2o


In [3]:
import tensorflow_datasets as tfds
import h2o
import numpy as np

In [4]:
h2o.init()
#started the h2o cluster.
#gpu cluster is nto required

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpga02cpqa
  JVM stdout: /tmp/tmpga02cpqa/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpga02cpqa/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.1
H2O_cluster_version_age:,3 months and 18 days !!!
H2O_cluster_name:,H2O_from_python_unknownUser_62c1jg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


h2o is more efficient than the scikitlearn 

https://www.quora.com/Why-would-one-use-H2O-ai-over-scikit-learn-machine-learning-tool
https://www.quora.com/Why-would-one-use-H2O-ai-over-scikit-learn-machine-learning-tool

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
dataset, info = tfds.load('amazon_us_reviews/Personal_Care_Appliances_v1_00',with_info=True,batch_size=-1,data_dir='/content/drive/My Drive/colabfiles')
train_dataset=dataset['train']

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /content/drive/My Drive/colabfiles/amazon_us_reviews/Personal_Care_Appliances_v1_00/0.1.0.incomplete519LG7/amazon_us_reviews-train.tfrecord


ERROR:absl:Statistics generation doesn't work for nested structures yet



Dataset amazon_us_reviews downloaded and prepared to /content/drive/My Drive/colabfiles/amazon_us_reviews/Personal_Care_Appliances_v1_00/0.1.0. Subsequent calls will reuse this data.


In [7]:
info

tfds.core.DatasetInfo(
    name='amazon_us_reviews',
    version=0.1.0,
    description='Amazon Customer Reviews (a.k.a. Product Reviews) is one of Amazons iconic products. In a period of over two decades since the first review in 1995, millions of Amazon customers have contributed over a hundred million reviews to express opinions and describe their experiences regarding products on the Amazon.com website. This makes Amazon Customer Reviews a rich source of information for academic researchers in the fields of Natural Language Processing (NLP), Information Retrieval (IR), and Machine Learning (ML), amongst others. Accordingly, we are releasing this data to further research in multiple disciplines related to understanding customer product experiences. Specifically, this dataset was constructed to represent a sample of customer evaluations and opinions, variation in the perception of a product across geographical regions, and promotional intent or bias in reviews.

Over 130+ million cus

In [8]:
dataset=tfds.as_numpy(train_dataset)

In [9]:
dataset

{'data': {'customer_id': array([b'13986323', b'50574716', b'50593972', ..., b'40719682',
         b'35596948', b'29430209'], dtype=object),
  'helpful_votes': array([0, 3, 0, ..., 0, 0, 0], dtype=int32),
  'marketplace': array([b'US', b'US', b'US', ..., b'US', b'US', b'US'], dtype=object),
  'product_category': array([b'Personal_Care_Appliances', b'Personal_Care_Appliances',
         b'Personal_Care_Appliances', ..., b'Personal_Care_Appliances',
         b'Personal_Care_Appliances', b'Personal_Care_Appliances'],
        dtype=object),
  'product_id': array([b'B00847JQZ6', b'B00N5HD340', b'B0077L1X24', ..., b'B000UZ8X2W',
         b'B000NURPPK', b'B001EY5GNW'], dtype=object),
  'product_parent': array([b'997683625', b'955577225', b'120764066', ..., b'96066145',
         b'58591097', b'986877728'], dtype=object),
  'product_title': array([b'SE - Reading Glass - Spring Loaded Hinges, 4.0x - RTS62400',
         b'Straight Razor',
         b'Philips Sonicare Flexcare & Healthy White Plastic

In [10]:
helpful_votes = dataset['data']['helpful_votes']
review_headline = dataset['data']['review_headline']
review_body = dataset['data']['review_body']
rating = dataset['data']['star_rating']

In [11]:
h2o_df = h2o.H2OFrame(np.hstack((helpful_votes[:,None],review_headline[:,None],review_body[:,None],rating[:,None])),column_names=['votes','headline','reviews','rating'],column_types=['numeric','string','string','numeric'])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [12]:
h2o_df

votes,headline,reviews,rating
0,b'These glasses are an excellent value. The fit is good and they are ...',"b""These glasses are an excellent value. The fit is good and they are very comfortable. Because of my legal blindness, there aren't a lot of options to try to see better, but I believe these help with my other visual aids, and because they are reasonably priced I can have more than one pair available.""",4
3,b'A fantastic way to cheaply try straight razor shaving.',"b""Always wanted to try straight razor shaving (as a DE safety razor user), and this was a cheap way for me to determine I was not into it.<br /><br />Because the blades are disposable and always sharp, I could put a new one in and reasonably rely upon that fact that cuts were probably due to my technique and not the blade.<br /><br />It's very hard to do straight razor shaving on yourself because the ANGLE is difficult to control without switching hands. Being very right-handed, I really couldn't do that. I bet I could shave someone else's face with it though.<br /><br />An immediate upside? Using a DE safety razor (slant edged even) seems SUPER safe now! I'm increased my speed with the DE due to that confidence, and I'd been using it for years now.""",5
0,b'Great for travel',"b'I usually either throw my toothbrush in a plastic bag with spare head so this product is very convenient for keeping all the parts apart, dry and undamaged, and i now keep it in my travel bag all the time ready to go.'",4
0,b'Five Stars',b'Top quality.',5
1,b'*Product sent not as shown',"b'Today I received 1 Fl. Oz, Natures Balance AlkaMAX alkaline booster liquid unflavored (formally TriMedica) a glass brown bottle with eye dropper.<br />Supplement Facts; Serving Size: 10 Drops (Approx. 0.5 mL). Servings per Container: Approx. 60<br />Amount Per Serving % Daily Value<br />Magnesium (as Magnesium Chloride) 40 mg 10%<br />Chloride (as Magnesium Chloride) 64 mg 2%<br />Other Ingredients: Deionized water and trace mineral complex.<br />Directions: Add 10 drops to 8 oz. of water or other beverage daily as needed or as directed by your health care practitioner. Store in a cool, dry place.<br /><br />I have not used product as of yet with changed ingredients of original/as shown in AlkaMax alkaline support white squeeze bottle.<br />but had to rate to post...heck this new may be better for all I know.'",3
6,b'Seems to work well!',"b'I recently picked this unit up at Bed and Bath where it was $99.00. With a 20% coupon the price was $80 (so comparison shop!) It\'s easy to use and seems to work well. I agree with other reviewers about the noise level (loud like a tea kettle boiling), but I use this unit in the living room area so sleeping is not an issue. Having the 2 separate water reservoirs to fill is much easier, weight-wise, as I previously had another humidifier which had one large water reservoir and it was very heavy when full. The unit comes apart quite easily for cleaning and since frequent cleaning is recommended (I usually do it 1x per week, not daily) this is a definite plus. Additionally, one only needs to use vinegar to clean the inside reservoir, which is a lot cheaper than having to buy some kind of \\\\""special\\\\"" cleaning solution. My one complaint is that it doesn\'t, but should, have an automatic shut off when both tanks are empty.I have to remember to turn the unit off when I leave the house. My previous humidifiers all had this auto turn off; seems like every humidifier should.'",4
8,b'deep heat?????',"b'Not sure what previous reviewer was talking about. The product uses ULTRASOUND waves for pain relief NOT heat. After repeated use, the pain in my knee and shoulder was feeling great. I think you should re-read the description and reconsider your opinion. Great product sold by a great company.'",5
2,b'Thermometer for a toddler',"b""The shippers were great....willing to consider speedier shipping options, but in the end the standard shipping arrived quickly enough. The 

In [13]:
h2o_df["target"] = (h2o_df["rating"] >= 4).ifelse("1","0")

In [14]:
h2o_df.shape

(85981, 5)

In [15]:
h2o_df["target"].table()
# from output we can see it is an imbalanced dataset positive vs negative wise

target,Count
0,23427
1,62554


In [16]:
h2o_df.describe()

Rows:85981
Cols:5




,votes,headline,reviews,rating,target
type,int,string,string,int,enum
mins,0.0,NaN,NaN,1.0,
mean,3.3494841883672,NaN,NaN,3.977401984159291,
maxs,2785.0,NaN,NaN,5.0,
sigma,19.483167220712946,NaN,NaN,1.433130291971473,
zeros,44555,0,0,0,
missing,0,0,0,0,0
0,0.0,b'These glasses are an excellent value. The fit is good and they are ...',"b""These glasses are an excellent value. The fit is good and they are very comfortable. Because of my legal blindness, there aren't a lot of options to try to see better, but I believe these help with my other visual aids, and because they are reasonably priced I can have more than one pair available.""",4.0,1
1,3.0,b'A fantastic way to cheaply try straight razor shaving.',"b""Always wanted to try straight razor shaving (as a DE safety razor user), and this was a cheap way for me to determine I was not into it.<br /><br />Because the blades are disposable and always sharp, I could put a new one in and reasonably rely upon that fact that cuts were probably due to my technique and not the blade.<br /><br />It's very hard to do straight razor shaving on yourself because the ANGLE is difficult to control without switching hands. Being very right-handed, I really couldn't do that. I bet I could shave someone else's face with it though.<br /><br />An immediate upside? Using a DE safety razor (slant edged even) seems SUPER safe now! I'm increased my speed with the DE due to that confidence, and I'd been using it for years now.""",5.0,1
2,0.0,b'Great for travel',"b'I usually either throw my toothbrush in a plastic bag with spare head so this product is very convenient for keeping all the parts apart, dry and undamaged, and i now keep it in my travel bag all the time ready to go.'",4.0,1


In [17]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
def tokenize(line):
  tokenized = line.tokenize("\\W+")
  tokenized = tokenized.tolower()
  tokenized = tokenized[(tokenized.nchar() >= 2) | (tokenized.isna()),:] #keep any value length of words greater than 2 ignore to , it etc. kind of words
  tokenized_filter = tokenized[(tokenized.isna()) | (~ tokenized.isin(stop_words)),:] #filter and remove if in the stop words list
  return tokenized_filter

In [19]:
words_reviews = tokenize(h2o_df["reviews"])

In [20]:
words_headline = tokenize(h2o_df["headline"])

In [21]:
words_reviews.head()

C1
glasses
excellent
value
fit
good
comfortable
legal
blindness
lot
options


In [22]:
words_headline.head()

C1
glasses
excellent
value
fit
good
""
fantastic
way
cheaply
try


In [23]:
words_reviews.shape

(2797601, 1)

In [24]:
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators import H2OGradientBoostingEstimator, H2OXGBoostEstimator

#word2vec model - words which share a close meaning or relation are stored closer in the vector
#better than tfidf vectors because in H2OWord2vecEstimator the context of the word is maintained

In [25]:
vec_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v_amazon.model")
vec_model.train(training_frame=words_reviews)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [39]:
h2o.save_model(vec_model,path='/content/drive/My Drive/colabfiles/',force=True)

'/content/drive/My Drive/colabfiles/w2v_amazon.model'

In [40]:
#specify the full path to model 
vec_model=h2o.load_model('/content/drive/My Drive/colabfiles/w2v_amazon.model')

In [41]:
#sanity check of model
#takes text corpus as input and produces word vector as output
vec_model.find_synonyms("perfumes",count=5)

OrderedDict([('scents', 0.7439516186714172),
             ('fragrances', 0.7290123105049133),
             ('woodsy', 0.724243700504303),
             ('weleda', 0.6952806115150452),
             ('patchouli', 0.694543719291687)])

In [42]:
vec_model.find_synonyms("toothpaste",count=5)

OrderedDict([('listerine', 0.7599316239356995),
             ('mouthwash', 0.7313197255134583),
             ('whitening', 0.7158499360084534),
             ('mentadent', 0.6744013428688049),
             ('ultreo', 0.6656889915466309)])

In [43]:
vec_model.find_synonyms("glasses",count=5)

OrderedDict([('sunglasses', 0.8881055116653442),
             ('readers', 0.8635859489440918),
             ('bifocal', 0.7921496033668518),
             ('lenses', 0.7885411977767944),
             ('bifocals', 0.7791802287101746)])

In [44]:
reviews_vecs = vec_model.transform(words_reviews,aggregate_method="AVERAGE")
#transform the words_reviews and transform to a vector representation as in a numeric representation that the model can understand. model cannot understand string objects
#aggregate = NONE , dont aggregate each input word is mapped to a single word vector
# average = input is treated as a sequence of words delimited by each sentence. each sentence is mapped to a vector. vector belonging to same sentence is averaged. look into more details.

In [45]:
reviews_vecs.shape

(85981, 100)

In [46]:
reviews_vecs.head()

C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0.159065,-0.0695679,-0.044262,-0.00451119,-0.10524,-0.120872,0.161854,0.0305692,0.101904,-0.0701404,0.0811984,-0.0183747,-0.0773768,-0.0464118,0.0758169,-0.0120015,0.101512,-0.0265924,-0.00569368,-0.119037,-0.061712,0.00695841,0.0711838,-0.0810358,-0.080483,-0.00602151,-0.0151043,-0.214798,0.188073,-0.049908,0.0672455,-0.0232842,-0.216857,0.162059,0.0861185,0.0304362,-0.0986123,0.0868728,0.0347802,-0.011445,-0.189832,0.0459568,0.00169208,0.0347999,-0.118835,-0.0071293,0.201103,0.0999017,0.0433043,-0.0553131,0.0583689,0.284984,-0.0125126,-0.0329167,-0.165511,0.0275919,-0.106101,0.0672348,0.00376391,-0.035072,-0.0626953,-0.0152396,0.0148236,-0.0311493,-0.119648,0.104999,-0.12311,0.18843,0.115969,0.0561582,-0.084873,-0.121456,-0.0669129,0.124051,0.0140443,-0.059215,0.135367,0.0530504,0.0822683,-0.0867747,-0.00211917,0.0264257,0.0175586,-0.0167558,-0.0719407,0.0620847,0.0464337,0.078521,0.0169472,0.139801,-0.0591891,-0.0649219,-0.0566241,0.182947,-0.0380743,-0.0614765,0.00551296,0.162098,0.0146094,0.0123741
-0.0121303,0.0715496,-0.148657,-0.0483579,-0.125889,-0.211522,-0.0400784,0.0241381,-0.0724646,-0.121377,-0.0476408,0.0541916,0.0370132,-0.0467518,0.0208398,0.000396439,0.0881359,0.1164,0.0280921,-0.0438392,-0.0518272,-0.0532945,0.041016,0.00336947,-0.134482,-0.125205,-0.135642,-0.055517,0.0102885,-0.00127493,0.112208,0.0877871,-0.211701,0.176113,0.00933897,0.0702944,-0.102127,0.0519632,-0.0487731,-0.140834,-0.290142,0.0508243,-0.0565894,0.120052,-0.00490817,0.170791,0.274651,-0.11375,-0.103787,-0.00436879,0.0609322,0.133775,-0.00364681,-0.0243833,-0.165821,0.0854756,-0.0772678,0.0451881,-0.0083775,-0.00961716,-0.0875445,-0.0279449,0.0690855,0.0201051,-0.139299,0.149993,0.0396791,0.00865685,0.147627,0.0873509,-0.0676782,0.00525626,-0.0821385,0.0264186,-0.124302,-0.0835362,0.0677949,-0.0580411,0.0245154,-0.0194159,-0.0946606,-0.0465931,-0.0555924,0.00483387,0.0151704,-0.0263208,0.0784847,0.192729,0.102259,0.100319,-0.0884916,0.00595638,-0.0268561,0.0270517,0.0573739,0.104121,0.0305728,0.120421,0.0671868,0.0852161
0.0423663,-0.054278,0.011334,0.103314,-0.133465,-0.0964777,-0.122465,-0.0645083,-0.0538524,-0.115683,-0.0056722,0.0935795,0.0178846,0.106558,-0.0393015,-0.00432922,0.0660326,-0.0211374,0.0510114,-0.237599,-0.155716,-0.000711579,0.112888,-0.0641811,-0.162616,-0.179745,0.0767009,-0.0137526,0.0818808,-0.0506365,0.0274429,0.0691674,-0.22985,0.0498593,0.0472129,-0.0904063,0.00912473,-0.101973,-0.0947424,-0.121249,-0.319479,0.106696,-0.0708721,0.128146,-0.0847446,0.0786706,-0.0532304,0.0625502,-0.0171913,0.0223831,-0.00874606,0.191131,-0.100159,-0.04063,-0.0394694,0.0136128,-0.162696,-0.0680124,0.0822946,-0.176956,-0.128832,-0.0298409,-0.00240394,0.0369105,-0.0332681,0.0718978,-0.063275,0.070579,0.124829,-0.0587965,0.000749187,-0.128934,-0.0158834,-0.0936429,-0.0672141,-0.12182,0.0937999,0.0264917,0.14068,-0.085169,-0.0406362,-0.00830762,0.0772125,0.0256574,-0.118631,-0.0310958,0.00988365,0.0528161,0.143076,0.118325,-0.179785,0.0726316,-0.119318,0.152957,0.0242713,0.0409947,0.099212,-0.0538118,0.0817105,0.0153966
-0.0491227,-0.0611696,-0.0719773,-0.0600334,-0.145317,-0.398135,0.0903167,0.187697,-0.120398,-0.278973,0.0650825,0.183099,0.0277089,-0.0668446,-0.156244,-0.0230193,0.208217,-0.0547922,0.185532,-0.282716,-0.0497108,-0.188823,0.0384947,-0.0018506,-0.254016,0.0251169,-0.0699478,-0.0798458,0.140281,-0.000557691,-0.217277,0.0386261,-0.121641,0.188629,0.153224,0.32332,-0.00128651,-0.0322262,-0.102995,-0.0498885,-0.162556,0.065664,0.207959,-0.139206,-0.17475,-0.144038,0.0171192,0.0176642,-0.166963,0.0250587,-0.0390203,0.0488041,0.0270663,-0

In [47]:
h2o_df_ext=h2o_df.cbind(reviews_vecs)

In [48]:
h2o_train,h2o_valid = h2o_df_ext.split_frame(ratios=[0.8])

In [49]:
h2o_valid['target'].table()

target,Count
0,4612
1,12469
